Import all necessary libraries:

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import ExcelWriter

Set up the crawler:

In [3]:
ticker = "MSFT" #important! use company ticker from yahoo finance here!
file_name = ticker + ".xlsx"
vfe = ["/financials?p=", "/balance-sheet?p=", "/cash-flow?p="]
df_list = []

base_url = "https://finance.yahoo.com/quote/"
urls = []

for statement in vfe:
    urls.append(base_url + ticker + statement + ticker)
        
for url in urls: #loop to get all financial data (BS, P&L, CF)
    page = requests.get(url)
    doc = BeautifulSoup(page.text, "html.parser")
    
    features = doc.find_all('div', class_='D(tbr)') #returns a list
    #print(features[0]) 
    #'div', class_='D(tbr)' is the class containing all other classes / data in each row (i.e. the overarching class)
    #features[0] = all data from first row; first row = header-row
    
    headers = []

    for item in features[0].find_all('div', class_='D(ib)'): #features[0] contains all data from first row; first row = header row
        headers.append(item.text)
    
    print(headers)
    
    index = 1 #start with second row since we already have data from header row
    data = []
    final = []
    while index <= len(features)-1:
    
        for item in features[index].find_all('div', class_='D(ib)'): #finds and extracts the label of each row
            data.append(item.text)
        for item in features[index].find_all('div', class_='Ta(c)'): #finds and extracts the data in each row
            data.append(item.text)
    
        final.append(data) # final has a lists-in-list structure (allows for easy transformation to pd.DataFrame)
        data = [] # clear data list after each row (i.e. after each features[index])
        index+=1
    print(final[:3]) #to check if the data from each statement is crawled correctly
    
    df = pd.DataFrame(final) #convert list 'final' to df
    df.columns = headers #specifiy column labels
    df.set_index('Breakdown') #set Breakdown column to be the index
    
    #function to make all values numerical
    def convert_to_numeric(column): #functions works as follows: a column is passed to the function and then firstly "," gets replaced, secondly "-" gets replaced
        first_col = [i.replace(',','') for i in column] #without replacing the comma, pd.to_numeric() returns an error
        second_col = [i.replace('-','') for i in first_col]
        final_col = pd.to_numeric(second_col)

        return final_col

    for column in headers[1:]: #1: because we do not want to convert Breakdown column
        df[column] = convert_to_numeric(df[column])
        
    df_list.append(df)
    
with pd.ExcelWriter(file_name) as writer:
    for n, df in enumerate(df_list): #
        df.to_excel(writer, 'sheet%s' % str(n + 1), index=False) # use n from for loop here, otherwise each sheet will be the same
    writer.save()
            

['Breakdown', 'ttm', '6/30/2019', '6/30/2018', '6/30/2017']
[['Total Revenue', '138,699,000', '125,843,000', '110,360,000', '89,950,000'], ['Cost of Revenue', '44,151,000', '42,910,000', '38,353,000', '34,261,000'], ['Gross Profit', '94,548,000', '82,933,000', '72,007,000', '55,689,000']]
['Breakdown', '6/30/2019', '6/30/2018', '6/30/2017']
[['Total Assets', '286,556,000', '258,848,000', '241,086,000'], ['Total Liabilities Net Minority Interest', '184,226,000', '176,130,000', '168,692,000'], ['Total Equity Gross Minority Interest', '102,330,000', '82,718,000', '72,394,000']]
['Breakdown', 'ttm', '6/30/2019', '6/30/2018', '6/30/2017']
[['Operating Cash Flow', '58,110,000', '52,185,000', '43,884,000', '39,507,000'], ['Investing Cash Flow', '-15,018,000', '-15,773,000', '-6,061,000', '-46,781,000'], ['Financing Cash Flow', '-42,455,000', '-36,887,000', '-33,590,000', '8,408,000']]
